In [1]:
!ls

Fetch_Data_Analysis.ipynb  data  logs  requirements.txt  src


In [2]:
import pandas as pd

In [3]:
#df = pd.read_json("users.json.gz", compression='gzip', lines=True) 
df = pd.read_json("data/users_sample_1.json") 
print(df.head())

                                    _id  active               createdDate  \
0  {'$oid': '5ff1e194b6a9d73a3a9f1052'}    True  {'$date': 1609687444800}   
1  {'$oid': '5ff1e194b6a9d73a3a9f1052'}    True  {'$date': 1609687444800}   
2  {'$oid': '5ff1e194b6a9d73a3a9f1052'}    True  {'$date': 1609687444800}   

                  lastLogin      role signUpSource state  
0  {'$date': 1609687537858}  consumer        Email    WI  
1  {'$date': 1609687537858}  consumer        Email    WI  
2  {'$date': 1609687537858}  consumer        Email    WI  


# **What are the requirements?**

    1. Review unstructured JSON data and diagram a new structured relational data model
    2. Generate a query that answers a predetermined business question
    3. Generate a query to capture data quality issues against the new structured relational data model
    4. Write a short email or Slack message to the business stakeholder


# Solution Approach.

**1. Data Infrastructure**

    - Create a docker to setup databricks runtime as the base image to process data using PySpark, Python and SQL.
    - Set up python and java environment variables just in case. 
    - Use Jupyter Notebook and data profiling and business queries. 

**2. Data Ingestion**

        - Ingest users JSON into a dataframe and create a "users" view. 
        - Ingest brands JSON into a dataframe and create a "brands" view. 
        - Ingest receipts JSON into a dataframe & create "receipts" and "receipts_item" table. 

**3. Data Profiling.**
        - Evaluate for null values
        - Evaluate for duplicates
        - Evaluate data inconsistencies and missing values. 

**4. Query to answer business questions**


In [4]:
import re
import json
import gzip
import pandas as pd

# Open the gzipped file
with gzip.open("data/users.json.gz", 'rt', encoding="utf-8") as f:
    # Read the file line by line
    data = []
    for line in f:
        # Try to load each line as a JSON object
        try:
            # Remove any trailing whitespace or newlines
            line = line.strip()
            if line:  # Skip empty lines
                json_object = json.loads(line + "\n")
                data.append(json_object)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line}, Error: {e}")

# Create the DataFrame from the collected JSON objects
#df = pd.DataFrame(data)
df = pd.DataFrame(pd.json_normalize(data))
print(df.head())

Skipping invalid JSON line: users.json                                                                                          0100777 0000000 0000000 00000254301 14567170355 010072  0                                                                                                    ustar 00                                                                                                                                                                                                                                                       {"_id":{"$oid":"5ff1e194b6a9d73a3a9f1052"},"active":true,"createdDate":{"$date":1609687444800},"lastLogin":{"$date":1609687537858},"role":"consumer","signUpSource":"Email","state":"WI"}, Error: Expecting value: line 1 column 1 (char 0)
Skipping invalid JSON line:                                                                                                                                                                                                     

## **Data Ingestion**

### **Ingest Users**

In [5]:
import json
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder.appName("UseJSONProcessing").getOrCreate()

# 1. Read the compressed NDJSON file as text. Spark automatically decompresses .gz files.
rdd = spark.sparkContext.textFile("data/users.json.gz")

# 2. Ensure each record ends with a newline character.
#    This transformation checks each line and appends "\n" if it's not present.
rdd_corrected = rdd.map(lambda line: line if line.endswith("\n") else line + "\n")


# 3. Parse the corrected RDD as JSON while capturing corrupted records.
#    We set the read mode to "PERMISSIVE" and define a column for corrupt records.
df_raw = spark.read.option("mode", "PERMISSIVE") \
                   .option("columnNameOfCorruptRecord", "_corrupt_record") \
                   .json(rdd_corrected)

# 4. Optional: Filter out the rows that contain corrupted records
df_valid = df_raw.filter(col("_corrupt_record").isNull())

# 5. Flatten the nested fields in the valid records:
#    - Extract the ObjectId from the _id field.
#    - Convert the createdDate and lastLogin fields from nested format to proper timestamps.
df_flattened = (df_valid.withColumn("_id", col("_id.$oid"))
                          .withColumn("createdDate", (col("createdDate.$date") / 1000).cast("timestamp"))
                          .withColumn("lastLogin", (col("lastLogin.$date") / 1000).cast("timestamp"))
                )

# 6. Show the resulting flattened DataFrame
# df_flattened.show(truncate=False)

# Optionally, log or inspect the corrupted records if needed:
corrupted_records = df_raw.filter(col("_corrupt_record").isNotNull())
if corrupted_records.count() > 0:
    print("Found corrupted records:")
    corrupted_records.show(truncate=False)

# 7. Register DataFrame as a SQL temporary view
# #  Users_raw and users views
df_raw.createOrReplaceTempView("users_raw")

# # # Register DataFrame as a SQL temporary view
df_flattened.createOrReplaceTempView("users")

# # # Run a Spark SQL query
df_users = spark.sql("SELECT _id,active,createdDate,lastLogin,role, signUpSource, state FROM users")


# # # Show the results
df_users.show(truncate=False)

df_users.describe().show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/22 17:08:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

Found corrupted records:
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

25/02/22 17:08:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+-----------+------------+-----+
|summary|                 _id|       role|signUpSource|state|
+-------+--------------------+-----------+------------+-----+
|  count|                 494|        494|         446|  438|
|   mean|                NULL|       NULL|        NULL| NULL|
| stddev|                NULL|       NULL|        NULL| NULL|
|    min|54943462e4b07e684...|   consumer|       Email|   AL|
|    max|60268c7aefa6011bb...|fetch-staff|      Google|   WI|
+-------+--------------------+-----------+------------+-----+



In [6]:
df_users.printSchema()

root
 |-- _id: string (nullable = true)
 |-- active: boolean (nullable = true)
 |-- createdDate: timestamp (nullable = true)
 |-- lastLogin: timestamp (nullable = true)
 |-- role: string (nullable = true)
 |-- signUpSource: string (nullable = true)
 |-- state: string (nullable = true)



### **Ingest Brands**

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder.appName("BrandsJSONProcessing").getOrCreate()

# 1. Read the compressed NDJSON file as text. Spark automatically decompresses .gz files.
rdd = spark.sparkContext.textFile("data/brands.json.gz")

# 2. Ensure each record ends with a newline character
rdd_corrected = rdd.map(lambda line: line if line.endswith("\n") else line + "\n")

# 3. Parse the corrected RDD as JSON
df = spark.read.json(rdd_corrected)

# 4. Flatten the nested fields:
#    - Extract the ObjectId from the _id field.
#    - Convert the createdDate and lastLogin fields from their nested format to proper timestamps
# Flatten the "cpg" nested field:
df_raw = (df.withColumn("_id", col("_id.$oid"))
            .withColumn("cpg_id", col("cpg.$id.$oid"))
            .withColumn("cpg_ref", col("cpg.$ref"))
             # .drop("cpg")
               )

# # 5. Show the resulting flattened DataFrame
df_raw.show(truncate=False)

# 7. Register DataFrame as a SQL temporary view
# #  brands_raw and brands views
df.createOrReplaceTempView("brands_raw")

# # # Register DataFrame as a SQL temporary view
df_raw.createOrReplaceTempView("brands")

# # # Run a Spark SQL query
df_brands = spark.sql("SELECT * FROM brands")

# # # Show the results
df_brands.show(truncate=False)

df_brands.describe().show()


25/02/22 17:28:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+------------------------+------------+-----------------------------+--------------------+--------------------+----------------------------------+-------------------------+--------+------------------------+-------+
|_id                     |barcode     |brandCode                    |category            |categoryCode        |cpg                               |name                     |topBrand|cpg_id                  |cpg_ref|
+------------------------+------------+-----------------------------+--------------------+--------------------+----------------------------------+-------------------------+--------+------------------------+-------+
|601ac115be37ce2ead437551|511111019862|NULL                         |Baking              |BAKING              |{{601ac114be37ce2ead437550}, Cogs}|test brand @1612366101024|false   |601ac114be37ce2ead437550|Cogs   |
|601c5460be37ce2ead43755f|511111519928|STARBUCKS                    |Beverages           |BEVERAGES           |{{5332f5fbe4b03c9a25efd0ba}, 

In [25]:
df_brands.printSchema()

root
 |-- _id: string (nullable = true)
 |-- barcode: string (nullable = true)
 |-- brandCode: string (nullable = true)
 |-- category: string (nullable = true)
 |-- categoryCode: string (nullable = true)
 |-- cpg: struct (nullable = true)
 |    |-- $id: struct (nullable = true)
 |    |    |-- $oid: string (nullable = true)
 |    |-- $ref: string (nullable = true)
 |-- name: string (nullable = true)
 |-- topBrand: boolean (nullable = true)
 |-- cpg_id: string (nullable = true)
 |-- cpg_ref: string (nullable = true)



### **Ingest Receipts**

In [26]:
import json
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, from_unixtime

# Initialize Spark Session
spark = SparkSession.builder.appName("ReceiptsJSONProcessing").getOrCreate()

# 1. Read the compressed NDJSON file as text. Spark automatically decompresses .gz files.
rdd = spark.sparkContext.textFile("data/receipts.json.gz")

# 2. Ensure each record ends with a newline character.
#    This transformation checks each line and appends "\n" if it's not present.
rdd_corrected = rdd.map(lambda line: line if line.endswith("\n") else line + "\n")

# # 3. Parse the corrected RDD as JSON
# df = spark.read.json(rdd_corrected)

# 3. Parse the corrected RDD as JSON while capturing corrupted records.
#    We set the read mode to "PERMISSIVE" and define a column for corrupt records.
df_raw = spark.read.option("mode", "PERMISSIVE") \
                   .option("columnNameOfCorruptRecord", "_corrupt_record") \
                   .json(rdd_corrected)

df_raw = df_raw.withColumn("_id", col("_id.$oid"))

# df_raw.show(truncate=False)

# # 4. Optional: Filter out the rows that contain corrupted records
# df_valid = df_raw.filter(col("_corrupt_record").isNull())

# Explode the "rewardsReceiptItemList" array so that each item becomes a separate row.
df_exploded = df_raw.withColumn("rewardsReceiptItem", explode("rewardsReceiptItemList"))

# 5. Flatten the nested fields in the valid records:
#    - Extract the ObjectId from the _id field.
#    - Convert the createdDate and lastLogin fields from nested format to proper timestamps.
#    - let's flatten the nested JSON columns in rewardsReceiptItem:
df_flattened = df_exploded.select(
    "_id",
    "bonusPointsEarned",
    "bonusPointsEarnedReason",
    from_unixtime(col("createDate.$date")/1000).alias("createDate"),
    from_unixtime(col("dateScanned.$date")/1000).alias("dateScanned"),
    from_unixtime(col("finishedDate.$date")/1000).alias("finishedDate"),
    from_unixtime(col("modifyDate.$date")/1000).alias("modifyDate"),
    from_unixtime(col("pointsAwardedDate.$date")/1000).alias("pointsAwardedDate"),
    col("pointsEarned"),
    from_unixtime(col("purchaseDate.$date")/1000).alias("purchaseDate"),
    "purchasedItemCount",
    col("rewardsReceiptStatus"),
    "totalSpent",
    "userId",
    "rewardsReceiptItemList"
    # Now flatten fields from rewardsReceiptItem
    # col("rewardsReceiptItem.barcode").alias("item_barcode"),
    # col("rewardsReceiptItem.description").alias("item_description"),
    # col("rewardsReceiptItem.finalPrice").alias("item_finalPrice"),
    # col("rewardsReceiptItem.itemPrice").alias("item_itemPrice"),
    # col("rewardsReceiptItem.needsFetchReview").alias("item_needsFetchReview"),
    # col("rewardsReceiptItem.partnerItemId").alias("item_partnerItemId"),
    # col("rewardsReceiptItem.preventTargetGapPoints").alias("item_preventTargetGapPoints"),
    # col("rewardsReceiptItem.quantityPurchased").alias("item_quantityPurchased"),
    # col("rewardsReceiptItem.userFlaggedBarcode").alias("item_userFlaggedBarcode"),
    # col("rewardsReceiptItem.userFlaggedNewItem").alias("item_userFlaggedNewItem"),
    # col("rewardsReceiptItem.userFlaggedPrice").alias("item_userFlaggedPrice"),
    # col("rewardsReceiptItem.userFlaggedQuantity").alias("item_userFlaggedQuantity")
)

# # 6. Show the resulting flattened DataFrame
# df_flattened.show(truncate=False)

# # Optionally, log or inspect the corrupted records if needed:
# corrupted_records = df_raw.filter(col("_corrupt_record").isNotNull())
# if corrupted_records.count() > 0:
#     print("Found corrupted records:")
#     corrupted_records.show(truncate=False)

# 7. Register DataFrame as a SQL temporary view
# #  Users_raw and users views
df_raw.createOrReplaceTempView("receipts_raw")

# # # Register DataFrame as a SQL temporary view
df_flattened.createOrReplaceTempView("receipts")

# # # Run a Spark SQL query
df_receipts = spark.sql("SELECT * FROM receipts")

# # # Show the results
# df_receipts.show(truncate=False)

df_receipts.describe().show()


25/02/22 17:31:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------+--------------------+------------------+-----------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+------------------+--------------------+
|summary|                 _id| bonusPointsEarned|bonusPointsEarnedReason|         createDate|        dateScanned|       finishedDate|         modifyDate|  pointsAwardedDate|      pointsEarned|       purchaseDate|purchasedItemCount|rewardsReceiptStatus|        totalSpent|              userId|
+-------+--------------------+------------------+-----------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+------------------+--------------------+
|  count|                6941|              5978|                   5978|               6941|               6941|        

In [10]:
df_raw.printSchema() 

root
 |-- _id: string (nullable = true)
 |-- bonusPointsEarned: long (nullable = true)
 |-- bonusPointsEarnedReason: string (nullable = true)
 |-- createDate: struct (nullable = true)
 |    |-- $date: long (nullable = true)
 |-- dateScanned: struct (nullable = true)
 |    |-- $date: long (nullable = true)
 |-- finishedDate: struct (nullable = true)
 |    |-- $date: long (nullable = true)
 |-- modifyDate: struct (nullable = true)
 |    |-- $date: long (nullable = true)
 |-- pointsAwardedDate: struct (nullable = true)
 |    |-- $date: long (nullable = true)
 |-- pointsEarned: string (nullable = true)
 |-- purchaseDate: struct (nullable = true)
 |    |-- $date: long (nullable = true)
 |-- purchasedItemCount: long (nullable = true)
 |-- rewardsReceiptItemList: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- barcode: string (nullable = true)
 |    |    |-- brandCode: string (nullable = true)
 |    |    |-- competitiveProduct: boolean (nullable = true)


In [11]:
df_receipts.printSchema()  

root
 |-- _id: string (nullable = true)
 |-- bonusPointsEarned: long (nullable = true)
 |-- bonusPointsEarnedReason: string (nullable = true)
 |-- createDate: string (nullable = true)
 |-- dateScanned: string (nullable = true)
 |-- finishedDate: string (nullable = true)
 |-- modifyDate: string (nullable = true)
 |-- pointsAwardedDate: string (nullable = true)
 |-- pointsEarned: string (nullable = true)
 |-- purchaseDate: string (nullable = true)
 |-- purchasedItemCount: long (nullable = true)
 |-- rewardsReceiptStatus: string (nullable = true)
 |-- totalSpent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- rewardsReceiptItemList: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- barcode: string (nullable = true)
 |    |    |-- brandCode: string (nullable = true)
 |    |    |-- competitiveProduct: boolean (nullable = true)
 |    |    |-- competitorRewardsGroup: string (nullable = true)
 |    |    |-- deleted: boolean (nullable = 

### **Create Receipt_Item view**

In [12]:
df_receipt_item_list = df_exploded.select(
    col("_id").alias("receipt_id"),
    from_unixtime(col("createDate.$date")/1000).alias("createDate"),
    from_unixtime(col("dateScanned.$date")/1000).alias("dateScanned"),
    from_unixtime(col("finishedDate.$date")/1000).alias("finishedDate"),
    from_unixtime(col("modifyDate.$date")/1000).alias("modifyDate"),
    from_unixtime(col("pointsAwardedDate.$date")/1000).alias("pointsAwardedDate"),
    from_unixtime(col("purchaseDate.$date")/1000).alias("purchaseDate"),
    col("userId"),
    col("totalSpent"),
    col("pointsEarned"),
    col("rewardsReceiptStatus"),
    col("rewardsReceiptItem.barcode").alias("item_barcode"),
    col("rewardsReceiptItem.brandCode").alias("item_brandCode"),
    col("rewardsReceiptItem.competitiveProduct").alias("item_competitiveProduct"),
    col("rewardsReceiptItem.competitorRewardsGroup").alias("item_competitorRewardsGroup"),
    col("rewardsReceiptItem.deleted").alias("item_deleted"),
    col("rewardsReceiptItem.description").alias("item_description"),
    col("rewardsReceiptItem.discountedItemPrice").alias("item_discountedItemPrice"),
    col("rewardsReceiptItem.finalPrice").alias("item_finalPrice"),
    col("rewardsReceiptItem.itemNumber").alias("item_itemNumber"),
    col("rewardsReceiptItem.itemPrice").alias("item_itemPrice"),
    col("rewardsReceiptItem.metabriteCampaignId").alias("item_metabriteCampaignId"),
    col("rewardsReceiptItem.needsFetchReview").alias("item_needsFetchReview"),
    col("rewardsReceiptItem.needsFetchReviewReason").alias("item_needsFetchReviewReason"),
    col("rewardsReceiptItem.originalFinalPrice").alias("item_originalFinalPrice"),
    col("rewardsReceiptItem.originalMetaBriteBarcode").alias("item_originalMetaBriteBarcode"),
    col("rewardsReceiptItem.originalMetaBriteDescription").alias("item_originalMetaBriteDescription"),
    col("rewardsReceiptItem.originalMetaBriteItemPrice").alias("item_originalMetaBriteItemPrice"),
    col("rewardsReceiptItem.originalMetaBriteQuantityPurchased").alias("item_originalMetaBriteQuantityPurchased"),
    col("rewardsReceiptItem.originalReceiptItemText").alias("item_originalReceiptItemText"),
    col("rewardsReceiptItem.partnerItemId").alias("item_partnerItemId"),
    col("rewardsReceiptItem.pointsEarned").alias("item_pointsEarned"),
    col("rewardsReceiptItem.pointsNotAwardedReason").alias("item_pointsNotAwardedReason"),
    col("rewardsReceiptItem.pointsPayerId").alias("item_pointsPayerId"),
    col("rewardsReceiptItem.preventTargetGapPoints").alias("item_preventTargetGapPoints"),
    col("rewardsReceiptItem.priceAfterCoupon").alias("item_priceAfterCoupon"),
    col("rewardsReceiptItem.quantityPurchased").alias("item_quantityPurchased"),
    col("rewardsReceiptItem.rewardsGroup").alias("item_rewardsGroup"),
    col("rewardsReceiptItem.rewardsProductPartnerId").alias("item_rewardsProductPartnerId"),
    col("rewardsReceiptItem.targetPrice").alias("item_targetPrice"),
    col("rewardsReceiptItem.userFlaggedBarcode").alias("item_userFlaggedBarcode"),
    col("rewardsReceiptItem.userFlaggedDescription").alias("item_userFlaggedDescription"),
    col("rewardsReceiptItem.userFlaggedNewItem").alias("item_userFlaggedNewItem"),
    col("rewardsReceiptItem.userFlaggedPrice").alias("item_userFlaggedPrice"),
    col("rewardsReceiptItem.userFlaggedQuantity").alias("item_userFlaggedQuantity")
)

# Show results
# df_receipt_item_list.show(truncate=False)

# # # Register DataFrame as a SQL temporary view
df_receipt_item_list.createOrReplaceTempView("receipt_item")

# # # Run a Spark SQL query
df_receipt_item = spark.sql("SELECT * FROM receipt_item")

# # # Show the results
# df_receipt_item.show(truncate=False)

In [13]:
df_receipt_item.describe().show(truncate=False, vertical=True)

[Stage 20:>                                                         (0 + 1) / 1]

-RECORD 0---------------------------------------------------------------------------------------------------------------
 summary                                 | count                                                                        
 receipt_id                              | 6941                                                                         
 createDate                              | 6941                                                                         
 dateScanned                             | 6941                                                                         
 finishedDate                            | 5967                                                                         
 modifyDate                              | 6941                                                                         
 pointsAwardedDate                       | 6078                                                                         
 purchaseDate                   

In [14]:
df_receipt_item.printSchema() 

root
 |-- receipt_id: string (nullable = true)
 |-- createDate: string (nullable = true)
 |-- dateScanned: string (nullable = true)
 |-- finishedDate: string (nullable = true)
 |-- modifyDate: string (nullable = true)
 |-- pointsAwardedDate: string (nullable = true)
 |-- purchaseDate: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- totalSpent: string (nullable = true)
 |-- pointsEarned: string (nullable = true)
 |-- rewardsReceiptStatus: string (nullable = true)
 |-- item_barcode: string (nullable = true)
 |-- item_brandCode: string (nullable = true)
 |-- item_competitiveProduct: boolean (nullable = true)
 |-- item_competitorRewardsGroup: string (nullable = true)
 |-- item_deleted: boolean (nullable = true)
 |-- item_description: string (nullable = true)
 |-- item_discountedItemPrice: string (nullable = true)
 |-- item_finalPrice: string (nullable = true)
 |-- item_itemNumber: string (nullable = true)
 |-- item_itemPrice: string (nullable = true)
 |-- item_metabrite

## **Evaluate Data Quality Issues in the Data Provided**

Using the programming language of your choice (SQL, Python, R, Bash, etc...) identify as many data quality issues as you can. We are not expecting a full blown review of all the data provided, but instead want to know how you explore and evaluate data of questionable provenance.

Commit your code and findings to the git repository along with the rest of the exercise


In [15]:
df_users.describe().show()

+-------+--------------------+-----------+------------+-----+
|summary|                 _id|       role|signUpSource|state|
+-------+--------------------+-----------+------------+-----+
|  count|                 494|        494|         446|  438|
|   mean|                NULL|       NULL|        NULL| NULL|
| stddev|                NULL|       NULL|        NULL| NULL|
|    min|54943462e4b07e684...|   consumer|       Email|   AL|
|    max|60268c7aefa6011bb...|fetch-staff|      Google|   WI|
+-------+--------------------+-----------+------------+-----+



#### **1. Formatting Issue:**

    - User data JSON does not conform to a valid JSON syntax - unable to read JSON lines as a single JSON document with line-delimited mode. 
        - Requires additional data clean up and enforcing to process line by line. 
                1. Read the file as a string
                2. Fix the structure (e.g., by inserting commas between objects, or wrapping the entire string in square brackets.
                3. Process data
        - Not recommended for large datasets and impact scalability and to enable streaming. 
    - 2 user records not processed due to invalid JSON format (corrupt records) 

In [16]:
# SQL query for the view
sql_query = """
    SELECT count(1) as corrupt_record_cnt FROM users_raw where _corrupt_record is not NULL
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)


+------------------+
|corrupt_record_cnt|
+------------------+
|2                 |
+------------------+



#### **2. Evaluate NULL values.**

        - Check NULLs in lastLogin, signUpSource and state fields in "user" table. 

In [17]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT 'Count of NULL values: NULL _id', count(1) as cnt FROM users where _id is NULL
    UNION
    SELECT 'Count of NULL values: NULL createdDate', count(1) as cnt FROM users where createdDate is NULL
    UNION
    SELECT 'Count of NULL values: NULL lastLogin', count(1) as cnt FROM users where lastLogin is NULL
    UNION
    SELECT 'Count of NULL values: NULL active', count(1) as cnt FROM users where active is NULL
    UNION
    SELECT 'Count of NULL values: NULL signUpSource', count(1) as cnt FROM users where signUpSource is NULL
    UNION
    SELECT 'Count of NULL values: NULL state', count(1) as cnt FROM users where state is NULL
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)

#df_trans.printSchema()  

+---------------------------------------+---+
|Count of NULL values: NULL _id         |cnt|
+---------------------------------------+---+
|Count of NULL values: NULL _id         |0  |
|Count of NULL values: NULL createdDate |0  |
|Count of NULL values: NULL lastLogin   |62 |
|Count of NULL values: NULL active      |0  |
|Count of NULL values: NULL signUpSource|48 |
|Count of NULL values: NULL state       |56 |
+---------------------------------------+---+



**b. Check NULLs in "brands" table.**

In [27]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT 'Count of NULL values: NULL barcode' as Count_of_NULLs, count(1) as cnt FROM brands where barcode is NULL
    UNION
    SELECT 'Count of NULL values: NULL brandCode' as Count_of_NULLs, count(1) as cnt FROM brands where brandCode is NULL
    UNION
    SELECT 'Count of NULL values: NULL category' as Count_of_NULLs, count(1) as cnt FROM brands where category is NULL
    UNION
    SELECT 'Count of NULL values: NULL categoryCode' as Count_of_NULLs, count(1) as cnt FROM brands where categoryCode is NULL
    UNION
    SELECT 'Count of NULL values: NULL name' as Count_of_NULLs, count(1) as cnt FROM brands where name is NULL
    UNION
    SELECT 'Count of NULL values: NULL cpg_id' as Count_of_NULLs, count(1) as cnt FROM brands where cpg_id is NULL
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)

#df_trans.printSchema()  

+---------------------------------------+---+
|Count_of_NULLs                         |cnt|
+---------------------------------------+---+
|Count of NULL values: NULL barcode     |0  |
|Count of NULL values: NULL brandCode   |234|
|Count of NULL values: NULL category    |155|
|Count of NULL values: NULL categoryCode|650|
|Count of NULL values: NULL name        |0  |
|Count of NULL values: NULL cpg_id      |0  |
+---------------------------------------+---+



**c. Check NULLs in "Receipts" table.** 

In [31]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT 'Count of NULL values: NULL _id' as Count_of_NULLs, count(1) as cnt FROM receipts where _id is NULL
    UNION
    SELECT 'Count of NULL values: NULL createDate' as Count_of_NULLs, count(1) as cnt FROM receipts where createDate is NULL
    UNION
    SELECT 'Count of NULL values: NULL finishedDate ' as Count_of_NULLs, count(1) as cnt FROM receipts where finishedDate is NULL
    UNION
    SELECT 'Count of NULL values: NULL purchaseDate' as Count_of_NULLs, count(1) as cnt FROM receipts where purchaseDate is NULL
    UNION
    SELECT 'Count of NULL values: NULL rewardsReceiptStatus' as Count_of_NULLs, count(1) as cnt FROM receipts where rewardsReceiptStatus is NULL
    UNION
    SELECT 'Count of NULL values: NULL userid' as Count_of_NULLs, count(1) as cnt FROM receipts where userid is NULL
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)


+-----------------------------------------------+---+
|Count_of_NULLs                                 |cnt|
+-----------------------------------------------+---+
|Count of NULL values: NULL _id                 |0  |
|Count of NULL values: NULL createDate          |0  |
|Count of NULL values: NULL finishedDate        |974|
|Count of NULL values: NULL purchaseDate        |21 |
|Count of NULL values: NULL rewardsReceiptStatus|0  |
|Count of NULL values: NULL userid              |0  |
+-----------------------------------------------+---+



**d. Check NULLs in "receipt_item" table.**

In [32]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT 'Count of NULL values: NULL receipt_id' as Count_of_NULLs, count(1) as cnt FROM receipt_item where receipt_id is NULL
    UNION
    SELECT 'Count of NULL values: NULL createDate' as Count_of_NULLs, count(1) as cnt FROM receipt_item where createDate is NULL
    UNION
    SELECT 'Count of NULL values: NULL dateScanned ' as Count_of_NULLs, count(1) as cnt FROM receipt_item where dateScanned is NULL
    UNION
    SELECT 'Count of NULL values: NULL userId' as Count_of_NULLs, count(1) as cnt FROM receipt_item where userId is NULL
    UNION
    SELECT 'Count of NULL values: NULL item_barcode' as Count_of_NULLs, count(1) as cnt FROM receipt_item where item_barcode is NULL
    UNION
    SELECT 'Count of NULL values: NULL item_brandCode' as Count_of_NULLs, count(1) as cnt FROM receipt_item where item_brandCode is NULL
    UNION
    SELECT 'Count of NULL values: NULL item_description' as Count_of_NULLs, count(1) as cnt FROM receipt_item where item_description is NULL
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)


+-------------------------------------------+----+
|Count_of_NULLs                             |cnt |
+-------------------------------------------+----+
|Count of NULL values: NULL receipt_id      |0   |
|Count of NULL values: NULL createDate      |0   |
|Count of NULL values: NULL dateScanned     |0   |
|Count of NULL values: NULL userId          |0   |
|Count of NULL values: NULL item_barcode    |3851|
|Count of NULL values: NULL item_brandCode  |4341|
|Count of NULL values: NULL item_description|381 |
+-------------------------------------------+----+



#### **3. Evaluate duplicates values in "user" table.** 

**Duplicates in User table**

    - See the result with duplicates and after applying distinct on user table for the ids, '600f47f06fd0dc1768a34a12','6014558767804a1228b20d00', '5ff5d15aeb7c7d12096d91a2

In [18]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT _id,  count(1) as cnt FROM users group by _id
    having count(1) > 1
"""
# Execute SQL query
df_user_profiling1 = spark.sql(sql_query)

df_user_profiling1.show(truncate=False)

+------------------------+---+
|_id                     |cnt|
+------------------------+---+
|600f47f06fd0dc1768a34a12|3  |
|60005709bd4dff11dda90ac1|2  |
|6014558767804a1228b20d00|2  |
|5fbc35711d967d1222cbfefc|3  |
|5ffc8f9704929111f6e922bf|6  |
|5fff55dabd4dff11dda8f5f1|7  |
|5ff4ce3dc3d63511e2a484dc|4  |
|6011f33173c60b1804ce1102|4  |
|5fff0f4fb3348b03eb45abb0|4  |
|60189c74c8b50e11d8454eff|7  |
|5ff73b90eb7c7d31ca8a452b|2  |
|601c2c05969c0b11f7d0b097|2  |
|60074b49325c8a1794623876|2  |
|600056a3f7e5b011fce897b0|8  |
|6008893b633aab121bb8e0a9|5  |
|6010bddaa4b74c120bd19dfb|3  |
|5ff5d15aeb7c7d12096d91a2|18 |
|60088d55633aab121bb8e41a|2  |
|6011f31ea4b74c18d3a8c476|4  |
|5ff36d0362fde912123a5535|3  |
+------------------------+---+
only showing top 20 rows



In [23]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT _id, active, createdDate, lastLogin, role, signUpSource, state FROM users 
      where _id in ('600f47f06fd0dc1768a34a12','6014558767804a1228b20d00', '5ff5d15aeb7c7d12096d91a2')
     
"""
# Execute SQL query
df_user_profiling1 = spark.sql(sql_query)

df_user_profiling1.show(truncate=False)

sql_query1 = """
    SELECT distinct _id, active, createdDate, lastLogin, role, signUpSource, state FROM users 
      where _id in ('600f47f06fd0dc1768a34a12','6014558767804a1228b20d00', '5ff5d15aeb7c7d12096d91a2')
     
"""
# Execute SQL query
df_user_profiling_dups = spark.sql(sql_query1)

df_user_profiling_dups.show(truncate=False)

#df_trans.printSchema()  

+------------------------+------+-----------------------+-----------------------+--------+------------+-----+
|_id                     |active|createdDate            |lastLogin              |role    |signUpSource|state|
+------------------------+------+-----------------------+-----------------------+--------+------------+-----+
|5ff5d15aeb7c7d12096d91a2|true  |2021-01-06 15:03:54.68 |2021-01-06 15:08:10.009|consumer|Email       |WI   |
|5ff5d15aeb7c7d12096d91a2|true  |2021-01-06 15:03:54.68 |2021-01-06 15:08:10.009|consumer|Email       |WI   |
|5ff5d15aeb7c7d12096d91a2|true  |2021-01-06 15:03:54.68 |2021-01-06 15:08:10.009|consumer|Email       |WI   |
|5ff5d15aeb7c7d12096d91a2|true  |2021-01-06 15:03:54.68 |2021-01-06 15:08:10.009|consumer|Email       |WI   |
|5ff5d15aeb7c7d12096d91a2|true  |2021-01-06 15:03:54.68 |2021-01-06 15:08:10.009|consumer|Email       |WI   |
|5ff5d15aeb7c7d12096d91a2|true  |2021-01-06 15:03:54.68 |2021-01-06 15:08:10.009|consumer|Email       |WI   |
|5ff5d15ae

In [20]:
df_brands.describe().show()

+-------+--------------------+--------------------+--------------------+--------+-------------+--------------------+
|summary|                 _id|             barcode|           brandCode|category| categoryCode|                name|
+-------+--------------------+--------------------+--------------------+--------+-------------+--------------------+
|  count|                1167|                1167|                 933|    1012|          517|                1167|
|   mean|                NULL|5.111114594175038...| 4.93037467029625E11|    NULL|         NULL|                NULL|
| stddev|                NULL|   287449.7490955448|9.476837353492757E10|    NULL|         NULL|                NULL|
|    min|5332f5ebe4b03c9a2...|        511111000167|                    |    Baby|         BABY|                   .|
|    max|6026d757be37ce636...|        511111919803|             ZUMBIDA|  Snacks|PERSONAL_CARE|test brand @16131...|
+-------+--------------------+--------------------+-------------

**Duplicates in Brands table**

    - For eg: Same barcode "511111004790" is associated with 2 products.

In [34]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT barcode,  count(1) as cnt FROM brands group by barcode
    having count(1) > 1
"""
# Execute SQL query
df_brands_profiling1 = spark.sql(sql_query)

df_brands_profiling1.show(truncate=False)

#df_trans.printSchema()  

+------------+---+
|barcode     |cnt|
+------------+---+
|511111204923|2  |
|511111504788|2  |
|511111004790|2  |
|511111605058|2  |
|511111305125|2  |
|511111704140|2  |
|511111504139|2  |
+------------+---+



In [35]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT _id, barcode, brandCode, category,categoryCode, name,topBrand,cpg_id,cpg_ref FROM brands 
    where barcode in ('511111204923', '511111504788', '511111004790', '511111605058', '511111305125', '511111704140', '511111504139')
    order by barcode
"""
# Execute SQL query
df_brands_profiling2 = spark.sql(sql_query)

df_brands_profiling2.show(truncate=False)

#df_trans.printSchema()  

+------------------------+------------+-------------+-------------------+------------+--------------------+--------+------------------------+-------+
|_id                     |barcode     |brandCode    |category           |categoryCode|name                |topBrand|cpg_id                  |cpg_ref|
+------------------------+------------+-------------+-------------------+------------+--------------------+--------+------------------------+-------+
|5c409ab4cd244a3539b84162|511111004790|ALEXA        |Baking             |NULL        |alexa               |true    |55b62995e4b0d8e685c14213|Cogs   |
|5cdacd63166eb33eb7ce0fa8|511111004790|BITTEN       |Condiments & Sauces|NULL        |Bitten Dressing     |NULL    |559c2234e4b06aca36af13c6|Cogs   |
|5c45f91b87ff3552f950f027|511111204923|0987654321   |Grocery            |NULL        |Brand1              |true    |5c45f8b087ff3552f950f026|Cogs   |
|5d6027f46d5f3b23d1bc7906|511111204923|CHESTERS     |Snacks             |NULL        |CHESTER'S     

#### **4. Evaluate other data anomalies** 

**1. Brand Name is present, but brandCode is NULL.**

In [36]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT _id, barcode, brandCode, category,categoryCode, name,topBrand,cpg_id,cpg_ref FROM brands 
    where brandCode is NULL
"""
# Execute SQL query
df_user_profiling1 = spark.sql(sql_query)

df_user_profiling1.show(truncate=False)

#df_trans.printSchema()  

+------------------------+------------+---------+--------+------------+-------------------------+--------+------------------------+-------+
|_id                     |barcode     |brandCode|category|categoryCode|name                     |topBrand|cpg_id                  |cpg_ref|
+------------------------+------------+---------+--------+------------+-------------------------+--------+------------------------+-------+
|601ac115be37ce2ead437551|511111019862|NULL     |Baking  |BAKING      |test brand @1612366101024|false   |601ac114be37ce2ead437550|Cogs   |
|57c08106e4b0718ff5fcb02c|511111102540|NULL     |NULL    |NULL        |MorningStar              |NULL    |5332f5f2e4b03c9a25efd0aa|Cpgs   |
|5fb28549be37ce522e165cb5|511111317364|NULL     |Baking  |BAKING      |test brand @1605535049181|false   |5fb28549be37ce522e165cb4|Cogs   |
|5332f5fee4b03c9a25efd0bd|511111303947|NULL     |NULL    |NULL        |Bottled Starbucks        |NULL    |53e10d6368abd3c7065097cc|Cpgs   |
|5332fa7ce4b03c9a25e

**2. Category is not NULL, categoryCode is NULL**

In [37]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT _id, barcode, brandCode, category,categoryCode, name,topBrand FROM brands 
    where (category is NULL and categoryCode is not NULL) or (category is not NULL and categoryCode is NULL)
"""
# Execute SQL query
df_user_profiling1 = spark.sql(sql_query)

df_user_profiling1.show(truncate=False)

#df_trans.printSchema()  

+------------------------+------------+----------------------------+--------------------+------------+--------------------------+--------+
|_id                     |barcode     |brandCode                   |category            |categoryCode|name                      |topBrand|
+------------------------+------------+----------------------------+--------------------+------------+--------------------------+--------+
|5cdad0f5166eb33eb7ce0faa|511111104810|J.L. KRAFT                  |Condiments & Sauces |NULL        |J.L. Kraft                |NULL    |
|5ab15636e4b0be0a89bb0b07|511111504412|CAMPBELLS HOME STYLE        |Canned Goods & Soups|NULL        |Campbell's Home Style     |false   |
|5c408e8bcd244a1fdb47aee7|511111504788|TEST                        |Baking              |NULL        |test                      |NULL    |
|588ba07be4b02187f85cdadd|511111201076|CALUMET                     |Baking              |NULL        |Calumet                   |false   |
|5d6413156d5f3b23d1bc790a|5

In [38]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT barcode, brandCode, count(1) FROM brands 
    group by barcode, brandCode
    having count(1) > 1
"""
# Execute SQL query
df_brands_prof1 = spark.sql(sql_query)

df_brands_prof1.show(truncate=False)

#df_trans.printSchema()  

+-------+---------+--------+
|barcode|brandCode|count(1)|
+-------+---------+--------+
+-------+---------+--------+



In [39]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT barcode, brandCode, count(1) FROM brands 
    group by barcode, brandCode
"""
# Execute SQL query
df_brands_prof1 = spark.sql(sql_query)

df_brands_prof1.show(truncate=False)

#df_trans.printSchema()  

+------------+-----------------------------+--------+
|barcode     |brandCode                    |count(1)|
+------------+-----------------------------+--------+
|511111117841|RAINBO                       |1       |
|511111300656|VASELINE                     |1       |
|511111200802|IMPERIAL                     |1       |
|511111901143|VELVEETA                     |1       |
|511111415787|TEST BRANDCODE @1598296704728|1       |
|511111904946|POP WORKS & COMPANY          |1       |
|511111817864|TEST BRANDCODE @1606765578435|1       |
|511111905523| TAZO BOTTLED TEAS           |1       |
|511111603641|NULL                         |1       |
|511111503255|NULL                         |1       |
|511111203605|NULL                         |1       |
|511111002154|GOOD SEASONS                 |1       |
|511111404880|BENIHANA                     |1       |
|511111905967|MOUNTAIN HIGH                |1       |
|511111806387|OATMEAL CRISP                |1       |
|511111500452|CERVEZA AGUILA

In [ ]:
# df_receipts.describe().show(truncate=False, vertical=True)


**3. Invalid barcode values in Receipts**

        - Junk or invalid barcode are associated to receipts. The barcode is 12 digit in length, but receipts have invalid barcode values. 

In [40]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT 'From Brands', length(barcode), count(1) FROM brands 
    group by 1, 2
    UNION
    SELECT 'From Receipts', length(item_barcode), count(1) FROM receipt_item     
    group by 1, 2
"""
# Execute SQL query
df_brands_prof1 = spark.sql(sql_query)

df_brands_prof1.show(truncate=False)

#df_trans.printSchema() 

+-------------+---------------+--------+
|From Brands  |length(barcode)|count(1)|
+-------------+---------------+--------+
|From Brands  |12             |1167    |
|From Receipts|13             |8       |
|From Receipts|11             |46      |
|From Receipts|10             |82      |
|From Receipts|12             |2619    |
|From Receipts|5              |2       |
|From Receipts|4              |332     |
|From Receipts|2              |1       |
|From Receipts|NULL           |3851    |
+-------------+---------------+--------+



**4: item_barcode vs item_brandCode anomaly**

        - Brand code is null in many cases. Barcode and Brandcode identifies the unique in brands data set. So, having NULL in brandcode will make receipts unable to join to brands for analysis. 
        

In [41]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT receipt_id, createDate, item_barcode,item_brandCode FROM receipt_item
    where item_brandCode is NULL or item_barcode is NULL
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)

+------------------------+-------------------+------------+--------------+
|receipt_id              |createDate         |item_barcode|item_brandCode|
+------------------------+-------------------+------------+--------------+
|5ff1e1eb0a720f0523000575|2021-01-03 15:25:31|4011        |NULL          |
|5ff1e1bb0a720f052300056b|2021-01-03 15:24:43|4011        |NULL          |
|5ff1e1bb0a720f052300056b|2021-01-03 15:24:43|028400642255|NULL          |
|5ff1e1f10a720f052300057a|2021-01-03 15:25:37|NULL        |NULL          |
|5ff1e1ee0a7214ada100056f|2021-01-03 15:25:34|4011        |NULL          |
|5ff1e1d20a7214ada1000561|2021-01-03 15:25:06|4011        |NULL          |
|5ff1e1d20a7214ada1000561|2021-01-03 15:25:06|1234        |NULL          |
|5ff1e1e40a7214ada1000566|2021-01-03 15:25:24|4011        |NULL          |
|5ff1e1cd0a720f052300056f|2021-01-03 15:25:01|NULL        |MISSION       |
|5ff1e1ed0a7214ada100056e|2021-01-03 15:25:33|4011        |NULL          |
|5ff1e1eb0a7214ada100056b

**5. Data Issue/Missing data: The count of items `rewardsReceiptItemList` and `purchasedItemCount` is not matching**

In [53]:
# Execute Spark SQL
# df_sales_owners = spark.sql("""
#     --with get_receipt as (
#     SELECT _id, createDate, userId, item_barcode, purchasedItemCount
#     FROM receipts
#     where purchasedItemCount = 689
#     """)

df_sales_owners = spark.sql("""
    SELECT _id as receipt_id, purchasedItemCount, count(1)
    FROM receipts
    where _id='600f2fc80a720f0535000030'
    group by 1, 2
    """)

# Show the result
df_sales_owners.show(truncate=False)

#df_3.show(truncate=False)

+------------------------+------------------+--------+
|receipt_id              |purchasedItemCount|count(1)|
+------------------------+------------------+--------+
|600f2fc80a720f0535000030|689               |459     |
+------------------------+------------------+--------+



In [55]:
# Execute Spark SQL
df_sales_owners = spark.sql("""
    SELECT 
        rewardsReceiptStatus, purchasedItemCount
    FROM receipts
    order by 2 desc
    """)

# Show the result
df_sales_owners.show(truncate=False)

df_sales_owners.createOrReplaceTempView("salesowners_vw")

#df_3.show(truncate=False)

+--------------------+------------------+
|rewardsReceiptStatus|purchasedItemCount|
+--------------------+------------------+
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
|FINISHED            |689               |
+--------------------+------------

### **Questions fom business stakeholders**

    1. What are the top 5 brands by receipts scanned for most recent month?
            - Use receipt Item table & brand table - brand name, receipts id, scanned date
            - Need to check if barcode and brandCode combo is unique or just barcode is required lookuo brand name. 
    2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
            - Use receipt Item table & brand table - brand name, receipts id, scanned date
            - Need to check if barcode and brandCode combo is unique or just barcode is required lookuo brand name.  
    3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
            - Use receipt table - rewardsReceiptStatus and totalspent columns. 
    4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
            - Use receipt table - rewardsReceiptStatus and purchasedItemCount columns
    5. Which brand has the most spend among users who were created within the past 6 months?
            - Use receipt Item table, user table abd brand table - brand name, receipts id, user create date
            - Need to check for active flag
    6. Which brand has the most transactions among users who were created within the past 6 months?
            - Use receipt Item table, user table abd brand table - brand name, receipts id, user create date
            - Need to check for active flag            

In [44]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT distinct receipt_id, item_barcode,item_brandCode, item_description FROM receipt_item
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)

+------------------------+------------+---------------------+---------------------------------------------------------------------------------+
|receipt_id              |item_barcode|item_brandCode       |item_description                                                                 |
+------------------------+------------+---------------------+---------------------------------------------------------------------------------+
|6004a5f20a7214ad4c000088|754500947807|HY-VEE               |Hy-Vee Lemon Flavored Lowfat Yogurt                                              |
|60049d9d0a720f05f3000094|039400015031|BUSH'S BEST          |Bush's Best Chili Beans - Kidney Beans in Mild Chili Sauce                       |
|600992f90a720f05fa000133|815473010360|BORDEN               |Borden 1% Chocolate Milk                                                         |
|600a1a8d0a7214ada2000008|612669316714|LAURA'S LEAN BEEF    |Laura's Lean Beef Beef - Ground                                            

In [45]:
# # Run a Spark SQL query 
# SQL query for the view
sql_query = """
    SELECT receipt_id, dateScanned, item_barcode,item_brandCode, item_description, bs.name, bs.category, bs.categoryCode
    FROM receipt_item ri inner join brands bs on nvl(ri.item_barcode,'~') = bs.barcode and nvl(ri.item_brandCode, '~') = nvl(bs.brandCode, '~')
    where item_barcode is not NULL
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)

+------------------------+-------------------+------------+---------------+----------------+----------------+--------------------+----------------------+
|receipt_id              |dateScanned        |item_barcode|item_brandCode |item_description|name            |category            |categoryCode          |
+------------------------+-------------------+------------+---------------+----------------+----------------+--------------------+----------------------+
|600a1e270a720f0535000009|2021-01-22 00:36:55|511111518044|SARGENTO       |SARGENTO        |Sargento® Cheese|Dairy & Refrigerated|DAIRY_AND_REFRIGERATED|
|600f2fc80a720f0535000030|2021-01-25 20:53:28|511111003960|MOUNTAIN DEW   |MOUNTAIN DEW    |Mountain Dew    |Beverages           |NULL                  |
|600f2fc80a720f0535000030|2021-01-25 20:53:28|511111101451|QUAKER         |QUAKER          |Quaker          |Breakfast & Cereal  |NULL                  |
|600a1a8d0a7214ada2000008|2021-01-22 00:21:32|511111101451|QUAKER         |Q

**Question**

    1. What are the top 5 brands by receipts scanned for most recent month?
            - Use receipt Item table & brand table - brand name, receipts id, scanned date
            - Need to check if barcode and brandCode combo is unique or just barcode is required lookuo brand name. 
    2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
            - Use receipt Item table & brand table - brand name, receipts id, scanned date
            - Need to check if barcode and brandCode combo is unique or just barcode is required lookuo brand name.              
  **Assessment**:
  
    For the  most recent month and prior month, in this case the recent month is "2021-03" and prior month is "2021-02", the query returms no value or null value due to data quality issues in "barcode" column in receipts. Incorrect or invalid barcode values causing the `brands` table and `receipts_item` join to fail and returns no value. 

The below table shows the list of top brand barcodes based on receipts scanned. Please review the "barcodes" here - none of the bar codes are valid.    

In [ ]:
 +--------------------------+------------+--------------+----------+
|month_label               |item_barcode|item_brandCode|item_count|
+--------------------------+------------+--------------+----------+
|Prior Recent month:2021-02|NULL        |NULL          |45        |
|Prior Recent month:2021-02|4011        |NULL          |33        |
|Prior Recent month:2021-02|B076FJ92M4  |NULL          |27        |
|Prior Recent month:2021-02|B07BRRLSVC  |NULL          |27        |
|Prior Recent month:2021-02|021000024674|NULL          |5         |
|Recent month:2021-03      |B076FJ92M4  |NULL          |13        |
|Recent month:2021-03      |B07BRRLSVC  |NULL          |13        |
+--------------------------+------------+--------------+----------+

`Get recent month. In this case the recent month is "2021-03", and prior month is "2021-02"`

In [49]:
sql_query = """
    select 
        min_dateScanned, year(min_dateScanned) as min_year, month(min_dateScanned) as min_month,  
        max_dateScanned, year(max_dateScanned) as max_year, month(max_dateScanned) as max_month
    from (
    SELECT min(dateScanned) min_dateScanned, max(dateScanned) max_dateScanned
    FROM receipt_item 
    )
"""
# Execute SQL query
df_top5_brands = spark.sql(sql_query)

df_top5_brands.show(truncate=False)

+-------------------+--------+---------+-------------------+--------+---------+
|min_dateScanned    |min_year|min_month|max_dateScanned    |max_year|max_month|
+-------------------+--------+---------+-------------------+--------+---------+
|2020-10-30 20:17:59|2020    |10       |2021-03-01 21:32:28|2021    |3        |
+-------------------+--------+---------+-------------------+--------+---------+



`Sample data to show the invalid barcodes`

In [50]:
sql_query = """
        SELECT * 
        FROM brands where (barcode like '%B07BRRLSVC%' or  barcode like '%B076FJ92M4%' or barcode like '%021000024674%')
        --in ('021000024674','B07BRRLSVC','B076FJ92M4')
"""

# Execute SQL query
df_top5_brands = spark.sql(sql_query)

df_top5_brands.show(truncate=False)

+---+-------+---------+--------+------------+---+----+--------+------+-------+
|_id|barcode|brandCode|category|categoryCode|cpg|name|topBrand|cpg_id|cpg_ref|
+---+-------+---------+--------+------------+---+----+--------+------+-------+
+---+-------+---------+--------+------------+---+----+--------+------+-------+



`Query to join receipt item and brands table to get the top 5 brands`

In [47]:
# Execute Spark SQL
df_sales_owners = spark.sql("""
    with get_latest_scan_date as 
    (
    select 
        min_dateScanned, year(min_dateScanned) as min_year, month(min_dateScanned) as min_month,  
        max_dateScanned, year(max_dateScanned) as max_year, month(max_dateScanned) as max_month
    FROM (
        SELECT min(dateScanned) min_dateScanned, max(dateScanned) max_dateScanned
        FROM receipt_item 
        )
    ), 
    top5_recent as 
    (
        SELECT bs.name as brand_name, item_barcode,item_brandCode, count(1) as item_count
        FROM receipt_item ri inner join brands bs 
                    on nvl(ri.item_barcode,'~') = bs.barcode and nvl(ri.item_brandCode, '~') = nvl(bs.brandCode, '~')
             CROSS JOIN  get_latest_scan_date
        where year(dateScanned) = max_year and month(dateScanned) = max_month
        GROUP BY 1,2,3
        ORDER BY item_count DESC
        LIMIT 5
        ),
    top5_prior as 
    (
        SELECT bs.name as brand_name,item_barcode,item_brandCode, count(1) as item_count
        FROM receipt_item ri inner join brands bs 
                    on nvl(ri.item_barcode,'~') = bs.barcode and nvl(ri.item_brandCode, '~') = nvl(bs.brandCode, '~')
             CROSS JOIN  get_latest_scan_date
        where year(dateScanned) = max_year and month(dateScanned) = max_month - 1
        GROUP BY 1,2,3
        ORDER BY item_count DESC
        LIMIT 5
        )
    SELECT * FROM top5_recent
    UNION
    SELECT * FROM top5_prior
    ;
        """)

# Show the result
df_sales_owners.show(truncate=False)

#df_3.show(truncate=False)

+----------+------------+--------------+----------+
|brand_name|item_barcode|item_brandCode|item_count|
+----------+------------+--------------+----------+
+----------+------------+--------------+----------+



`Query to get the top 5 brands based on receipts scanned from receipt item table without joining receipt item and brands table`

In [48]:
# where formatted_date >= add_months(current_date(), -12)
# sql_query = """
#     SELECT count(1) 
#     FROM receipt_item where year(dateScanned) =2021 and month(dateScanned) = 3
#     and item_barcode is not NULL
# """

sql_query = """
    WITH recent_month AS (
        SELECT 'Recent month:2021-03' AS month_label, item_barcode, item_brandCode, COUNT(1) AS item_count
        FROM receipt_item ri 
        WHERE YEAR(dateScanned) = 2021 AND MONTH(dateScanned) = 3 
        GROUP BY item_barcode, item_brandCode 
        ORDER BY item_count DESC
        LIMIT 5
    ), prior_month AS (
        SELECT 'Prior Recent month:2021-02' AS month_label, item_barcode, item_brandCode, COUNT(1) AS item_count
        FROM receipt_item ri 
        WHERE YEAR(dateScanned) = 2021 AND MONTH(dateScanned) = 2 
        GROUP BY item_barcode, item_brandCode 
        ORDER BY item_count DESC
        LIMIT 5
    )
    SELECT * FROM recent_month
    UNION ALL
    SELECT * FROM prior_month
    ORDER BY 1;
"""

# Execute SQL query
df_top5_brands = spark.sql(sql_query)

df_top5_brands.show(truncate=False)

+--------------------------+------------+--------------+----------+
|month_label               |item_barcode|item_brandCode|item_count|
+--------------------------+------------+--------------+----------+
|Prior Recent month:2021-02|NULL        |NULL          |45        |
|Prior Recent month:2021-02|4011        |NULL          |33        |
|Prior Recent month:2021-02|B076FJ92M4  |NULL          |27        |
|Prior Recent month:2021-02|B07BRRLSVC  |NULL          |27        |
|Prior Recent month:2021-02|021000024674|NULL          |5         |
|Recent month:2021-03      |B076FJ92M4  |NULL          |13        |
|Recent month:2021-03      |B07BRRLSVC  |NULL          |13        |
+--------------------------+------------+--------------+----------+



**Question**

    3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
        - Use receipt table - rewardsReceiptStatus and totalspent columns. 
    4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
        - Use receipt table - rewardsReceiptStatus and purchasedItemCount columns    
  **Assmuption**: There is no status "Accepted", instead "FINISHED" statusis used as "accepted" status.
  
  **Assessment**:
  
   1. Average spend for "FINISHED" status is greater than that of "REJECTED" status, `15.86046511627907` v/s `2.5441176470588234` 
   2. Total number of items purchased from each receipts for "FINISHED" status is greater than that of "REJECTED" status, `8184` v/s `173`
    

`Get a list of distinct rewardsReceiptStatus values`

In [51]:
# Execute Spark SQL
df_sales_owners = spark.sql("""
    --with get_receipt as (
    SELECT 
        rewardsReceiptStatus,
        sum(totalSpent) as total_spend, 
        avg(totalSpent) as average_spend
    FROM receipts
    group by rewardsReceiptStatus
    order by 3 desc
    """)

# Show the result
df_sales_owners.show(truncate=False)

#df_3.show(truncate=False)

+--------------------+------------------+------------------+
|rewardsReceiptStatus|total_spend       |average_spend     |
+--------------------+------------------+------------------+
|FLAGGED             |2134811.9000000213|2635.5702469136068|
|FINISHED            |7366687.770000038 |1244.7934724569175|
|PENDING             |1373.5900000000001|28.03244897959184 |
|REJECTED            |3264.0100000000025|19.902500000000014|
+--------------------+------------------+------------------+



In [52]:
# Execute Spark SQL
df_sales_owners = spark.sql("""
    with get_receipt as (
    select distinct _id as receipt_id, rewardsReceiptStatus, purchasedItemCount
    FROM receipts) 
    SELECT 
        rewardsReceiptStatus,
        sum(purchasedItemCount) as total_number_of_items_purchased, 
        avg(purchasedItemCount) as avg_of_items_purchased
    FROM get_receipt
    group by rewardsReceiptStatus
    order by 2 desc
    """)

# Show the result
df_sales_owners.show(truncate=False)

#df_3.show(truncate=False)

+--------------------+-------------------------------+----------------------+
|rewardsReceiptStatus|total_number_of_items_purchased|avg_of_items_purchased|
+--------------------+-------------------------------+----------------------+
|FINISHED            |8184                           |15.86046511627907     |
|FLAGGED             |1014                           |22.043478260869566    |
|REJECTED            |173                            |2.5441176470588234    |
|PENDING             |NULL                           |NULL                  |
+--------------------+-------------------------------+----------------------+



**Question**

    5. Which brand has the most spend among users who were created within the past 6 months?
        - Use receipt Item table, user table abd brand table - brand name, receipts id, user create date
        - Need to check for active flag
    6. Which brand has the most transactions among users who were created within the past 6 months?
        - Use receipt Item table, user table abd brand table - brand name, receipts id, user create date
        - Need to check for active flag       
  
  **Assessment**:
  
   1. `Tostitos` and `Swanson` are the top brand based on users spend among users who were created within the past 6 months (7527.79 vs 7187.14)
   2. `Tostitos` and `Swanson` are the tied at top based on most users transactions among users who were created within the past 6 months (11 each)
   3. Top 5 brand based on users spend and users transactions are given below table.
    

In [ ]:
+---------------------+------------+---------------+-----------------+
|brand_name           |item_barcode|item_brandCode |total_user_spend |
+---------------------+------------+---------------+-----------------+
|Tostitos             |511111001485|TOSTITOS       |7527.79          |
|Swanson              |511111204206|SWANSON        |7187.14          |
|Cracker Barrel Cheese|511111802358|CRACKER BARREL |4885.889999999999|
|Cheetos              |511111001768|CHEETOS        |4721.95          |
|Pepperidge Farm      |511111904175|PEPPERIDGE FARM|4721.95          |
+---------------------+------------+---------------+-----------------+

 +---------------------+------------+--------------+---------------------+
|brand_name           |item_barcode|item_brandCode|total_receipt_txn_cnt|
+---------------------+------------+--------------+---------------------+
|Tostitos             |511111001485|TOSTITOS      |11                   |
|Swanson              |511111204206|SWANSON       |11                   |
|Kettle Brand         |511111104537|KETTLE BRAND  |3                    |
|Cracker Barrel Cheese|511111802358|CRACKER BARREL|2                    |
|Cheetos              |511111001768|CHEETOS       |1                    |
+---------------------+------------+--------------+---------------------+

`Get 6 month date for users createdDate`

In [56]:
# # Run a Spark SQL query 
sql_query = """
    SELECT min(createdDate), max(createdDate), max(date(createdDate)), add_months(max(date(createdDate)),-6) FROM users
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)

#df_trans.printSchema()  

+-----------------------+----------------------+----------------+--------------------------------+
|min(createdDate)       |max(createdDate)      |max(createdDate)|add_months(max(createdDate), -6)|
+-----------------------+----------------------+----------------+--------------------------------+
|2014-12-19 14:21:22.381|2021-02-12 14:11:06.24|2021-02-12      |2020-08-12                      |
+-----------------------+----------------------+----------------+--------------------------------+



In [57]:
# # Run a Spark SQL query 
sql_query = """
    SELECT year(createdDate) year, month(createdDate) month, count(1) FROM users
    group by 1,2
    order by 1 desc, 2 desc
"""
# Execute SQL query
df_user_profiling = spark.sql(sql_query)

df_user_profiling.show(truncate=False)

#df_trans.printSchema() 

+----+-----+--------+
|year|month|count(1)|
+----+-----+--------+
|2021|2    |46      |
|2021|1    |354     |
|2020|12   |20      |
|2020|11   |24      |
|2020|7    |1       |
|2020|1    |1       |
|2017|12   |8       |
|2017|9    |18      |
|2017|7    |1       |
|2015|4    |1       |
|2014|12   |20      |
+----+-----+--------+



`Query to get top 5 brand based on users spend and users transactions.`

In [59]:
# Execute Spark SQL
df_user_txns = spark.sql("""
    with user_dates as 
    (
    SELECT min(createdDate) as min_createdate, 
        max(createdDate) as max_createdate, 
        max(date(createdDate)) as max_createdate_1, 
        add_months(max(date(createdDate)),-6) as 6months_date
    FROM users
    ),
    get_users as 
    (    
        SELECT    
            ur._id as userid, ur.state, ur.createdDate,ur.lastLogin,ur.role, ur.active
        FROM users ur, user_dates
        where date(ur.createdDate) >=  user_dates.6months_date
    ),
    user_txns_6months
    (
     SELECT  distinct 
             bs.name as brand_name, 
             item_barcode,
             item_brandCode, 
             receipt_id,  
             totalspent
        FROM receipt_item ri inner join brands bs 
                    on nvl(ri.item_barcode,'~') = bs.barcode and nvl(ri.item_brandCode, '~') = nvl(bs.brandCode, '~')
             inner join get_users gu on ri.userId = gu. userid   
    )
        select 
            brand_name, 
            item_barcode, 
            item_brandCode, 
            count(receipt_id) as total_receipt_txn_cnt, sum(totalspent) as total_user_spend
        from user_txns_6months
        GROUP BY 1,2,3
        ORDER BY total_receipt_txn_cnt DESC, total_user_spend desc
        LIMIT 5
        """)

# Show the result
df_user_txns.show(truncate=False)

#df_3.show(truncate=False)

+---------------------+------------+--------------+---------------------+-----------------+
|brand_name           |item_barcode|item_brandCode|total_receipt_txn_cnt|total_user_spend |
+---------------------+------------+--------------+---------------------+-----------------+
|Tostitos             |511111001485|TOSTITOS      |11                   |7527.79          |
|Swanson              |511111204206|SWANSON       |11                   |7187.14          |
|Kettle Brand         |511111104537|KETTLE BRAND  |3                    |2400.91          |
|Cracker Barrel Cheese|511111802358|CRACKER BARREL|2                    |4885.889999999999|
|Prego                |511111704140|PREGO         |1                    |4721.95          |
+---------------------+------------+--------------+---------------------+-----------------+

